# Guardar y cargar el modelo - 02 Cargar los pesos

## Cargar los pesos del modelo

Para cargar pesos de modelo, primero debe crear una instancia del mismo modelo, es decir, tenemos que volver a crear la red.

In [1]:
from torch import nn

# Creamos la red neuronal desde cero
class NeuralNetworkFromScratch(nn.Module):
    def __init__(self):
        super(NeuralNetworkFromScratch, self).__init__()   # Se inicializa el módulo nn.Module
        self.flatten = nn.Flatten()             # Se crea una primera capa que aplana la imagen de entrada
        self.linear_relu_stack = nn.Sequential( # Se crea una módulo de arquitectura secuencial:
            nn.Linear(3*32*32, 512),                # Se añade una primera capa lineal que está preparada 
                                                    # para que le entre un vector de 28*28 (784)
                                                    # y sacará un vector de 512
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 512),                    # Se añade una segunda capa lineal que le entran 512 
                                                    # datos y saca 512 datos
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 10)                      # Se añade una tercera capa lineal que le entran 512 
                                                    # datos y saca un array de tamaño 10 (el número
                                                    # de etiquetas)
        )
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)                         # Se pasa la imagen por la capa de aplanado
        logits = self.linear_relu_stack(x)          # Se pasa el vector resultante por la red
        #probs = self.softmax(logits)
        return logits

model_scratch = NeuralNetworkFromScratch()

Una vez tenemos la red creada, podemos cargar los parámetros usando el método ``load_state_dict()``.

In [2]:
from torch import load

path = "data/pesos.pth"
model_scratch.load_state_dict(load(path))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

 > **Nota**:
 > asegurarse de poner el modelo en modo evaluación llamando al método ``model.eval()`` antes de hacer inferencias para configurar las capas de normalización por lotes y de abandono en el modo de evaluación. No hacer esto producirá resultados de inferencia inconsistentes.

In [3]:
model_scratch.eval()

NeuralNetworkFromScratch(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

## Evaluar el modelo cargado

Nos descargamos el dataset de test y evaluamos el modelo

In [4]:
from torchvision import datasets
from torchvision.transforms import ToTensor

dataset_test = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Files already downloaded and verified


Se crea el dataloader

In [5]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

test_dataloader = DataLoader(dataset_test, batch_size=BATCH_SIZE)

Se crea el bucle de test

In [6]:
import torch

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Se lleva la red a la GPU, si la hay

In [7]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model_scratch.to(device)

Using cpu device


NeuralNetworkFromScratch(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

Se crea una función de pérdida

In [8]:
loss_fn = nn.CrossEntropyLoss()

Se comprueba el desempeño de la red en el dataset de test

In [9]:
test_loop(test_dataloader, model_scratch, loss_fn)

Test Error: 
 Accuracy: 10.4%, Avg loss: 2.299852 



Obtenemos métricas similares a cuando creamos y entrenamos la red